# Pydantic V2 Tutorial
## Class 1

In [1]:
import pydantic
print(pydantic.__version__)

2.10.6


In [2]:
class User:
    def __init__(self, id, name='Jane Doe'):
        if not isinstance(id, int):
            raise TypeError(f'Expected id to be an int, got {type(id).__name__}')
        
        if not isinstance(name, str):
            raise TypeError(f'Expected name to be a str, got {type(name).__name__}')
        
        self.id = id
        self.name = name

try:
    user = User(id='123')
except TypeError as e:
    print(e)

Expected id to be an int, got str


In [3]:
from pydantic import BaseModel

class User(BaseModel):
    id: int
    name: str = 'Jane Doe'

try:
    user = User(id='123') 
except pydantic.error_wrappers.ValidationError as e:
    print(e)

    

In [7]:
from pydantic import ValidationError

try:
    user = User(id='err') 
except ValidationError as e:
    print(e)


1 validation error for User
id
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='err', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/int_parsing


In [8]:
print(user.model_fields_set)

{'id'}


In [9]:
print(user.model_computed_fields)

{}


In [10]:
print(user.model_fields)

{'id': FieldInfo(annotation=int, required=True), 'name': FieldInfo(annotation=str, required=False, default='Jane Doe')}


In [14]:
from pprint import pprint
print(user.model_dump())
print("Model Dump with skip_defaults=False")
print(user.model_dump(exclude_defaults=False))
print("-----------------")
print("Model Dump JSON")
print(user.model_dump_json())
print("Model JSON schema")
pprint(user.model_json_schema())

{'id': 123, 'name': 'Jane Doe'}
Model Dump with skip_defaults=False
{'id': 123, 'name': 'Jane Doe'}
-----------------
Model Dump JSON
{"id":123,"name":"Jane Doe"}
Model JSON schema
{'properties': {'id': {'title': 'Id', 'type': 'integer'},
                'name': {'default': 'Jane Doe',
                         'title': 'Name',
                         'type': 'string'}},
 'required': ['id'],
 'title': 'User',
 'type': 'object'}


## Nested Models

In [16]:
from typing import List, Optional
from pydantic import BaseModel


class Food(BaseModel):
    name: str
    price: float
    ingredients: Optional[List[str]] = None


class Restaurant(BaseModel):
    name: str
    location: str
    foods: List[Food]


restaurant_instance = Restaurant(
    name="Tasty Bites",
    location="123, Flavor Street",
    foods=[
        {"name": "Cheese Pizza", "price": 12.50, "ingredients": ["Cheese", "Tomato Sauce", "Dough"]},
        {"name": "Veggie Burger", "price": 8.99}
    ]
)

print(restaurant_instance)
pprint(restaurant_instance.model_dump())

name='Tasty Bites' location='123, Flavor Street' foods=[Food(name='Cheese Pizza', price=12.5, ingredients=['Cheese', 'Tomato Sauce', 'Dough']), Food(name='Veggie Burger', price=8.99, ingredients=None)]
{'foods': [{'ingredients': ['Cheese', 'Tomato Sauce', 'Dough'],
            'name': 'Cheese Pizza',
            'price': 12.5},
           {'ingredients': None, 'name': 'Veggie Burger', 'price': 8.99}],
 'location': '123, Flavor Street',
 'name': 'Tasty Bites'}


In [17]:
!pip install pydantic[email]

  Using cached email_validator-2.2.0-py3-none-any.whl.metadata (25 kB)
Using cached email_validator-2.2.0-py3-none-any.whl (33 kB)


`conlist` is a function provided by the Pydantic library in Python. It is used to create a constrained list type, which allows you to specify certain constraints on the list, such as minimum and maximum length, and the type of items the list can contain.

Here's a breakdown of what `conlist` does and how it works:

- **Type of items**: You can specify the type of items that the list should contain. For example, `conlist(int)` creates a list that should only contain integers.
- **Minimum length**: You can specify the minimum number of items the list should contain using the `min_length` parameter.
- **Maximum length**: You can specify the maximum number of items the list should contain using the `max_length` parameter.

Here's an example of how to use `conlist`:

```python
from pydantic import BaseModel, conlist

class Employee(BaseModel):
    name: str
    position: str

# Define a constrained list type for a list of Employees with a minimum length of 2
EmployeeList = conlist(Employee, min_length=2)

class Company(BaseModel):
    employees: EmployeeList

# Example usage
company = Company(employees=[
    Employee(name="Alice", position="Developer"),
    Employee(name="Bob", position="Manager")
])

print(company)
```

In this example:
- `Employee` is a Pydantic model representing an employee.
- `EmployeeList` is a constrained list type that requires the list to contain at least 2 `Employee` objects.
- `Company` is a Pydantic model that uses `EmployeeList` as the type for its `employees` attribute.

By using `conlist`, you can enforce constraints on the list, ensuring that it meets the specified requirements.

In [20]:
from typing import List
from pydantic import BaseModel, EmailStr, PositiveInt, conlist, Field, HttpUrl

class Address(BaseModel):
    street: str
    city: str
    state: str
    zip_code: str

class Employee(BaseModel):
    name: str
    position: str
    email: EmailStr

# Define a new type for a list of Employees with a minimum length of 2
EmployeeList = conlist(Employee, min_length=2)

class Owner(BaseModel):
    name: str
    email: EmailStr
    
class Restaurant(BaseModel):
    name: str = Field(..., pattern=r"^[a-zA-Z0-9-' ]+$")
    owner: Owner
    address: Address
    employees: EmployeeList
    number_of_seats: PositiveInt
    delivery: bool
    website: HttpUrl

In [22]:
# Creating an instance of the Restaurant class
restaurant_instance = Restaurant(
    name="Tasty Bites",
    owner={
        "name": "John Doe",
        "email": "john.doe@example.com"
    },
    address={
        "street": "123, Flavor Street",
        "city": "Tastytown",
        "state": "TS",
        "zip_code": "12345",
    },
    employees=[
        {
            "name": "Jane Doe",
            "position": "Chef",
            "email": "jane.doe@example.com"
        },
        {
            "name": "Mike Roe",
            "position": "Waiter",
            "email": "mike.roe@example.com"
        }
    ],
    number_of_seats=50,
    delivery=True,
    website="http://tastybites.com"
)

# Printing the instance
pprint(restaurant_instance.model_dump())

{'address': {'city': 'Tastytown',
             'state': 'TS',
             'street': '123, Flavor Street',
             'zip_code': '12345'},
 'delivery': True,
 'employees': [{'email': 'jane.doe@example.com',
                'name': 'Jane Doe',
                'position': 'Chef'},
               {'email': 'mike.roe@example.com',
                'name': 'Mike Roe',
                'position': 'Waiter'}],
 'name': 'Tasty Bites',
 'number_of_seats': 50,
 'owner': {'email': 'john.doe@example.com', 'name': 'John Doe'},
 'website': HttpUrl('http://tastybites.com/')}


## Field Validators

In [24]:
from pydantic import BaseModel, Field, EmailStr, field_validator

class Owner(BaseModel):
    name: str
    email: EmailStr

    @field_validator("name")
    @classmethod
    def name_must_contain_space(cls, v):
        if ' ' not in v:
            raise ValueError('Owner name must contain a space')
        return v.title()
    

try:
    owner = Owner(name="JohnDoe", email="john@email.com")
except ValidationError as e:
    print(e)

1 validation error for Owner
name
  Value error, Owner name must contain a space [type=value_error, input_value='JohnDoe', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error


In [25]:
try:
    owner = Owner(name="omar hosney", email="john@email.com")
except ValidationError as e:
    print(e)

print(owner.name)

Omar Hosney


## Model validators
In Pydantic, the `@model_validator` decorator is used to define custom validation logic for a model. This allows you to perform additional checks and transformations on the data after the standard validation has been performed. The `@model_validator` decorator can be applied to methods within a Pydantic model class to enforce custom validation rules.

Here's a breakdown of how `@model_validator` works:

- **Custom validation**: You can define custom validation logic that runs after the standard Pydantic validation.
- **Class method**: The method decorated with `@model_validator` should be a class method, meaning it takes `cls` as its first parameter.
- **Post-validation**: The custom validation method is called after the standard validation has been completed.

Here's an example to illustrate the use of `@model_validator`:

```python
from pydantic import BaseModel, validator, ValidationError

class User(BaseModel):
    username: str
    age: int

    @validator('age')
    def check_age(cls, value):
        if value < 18:
            raise ValueError('Age must be at least 18')
        return value

    @validator('username')
    def check_username(cls, value):
        if not value.isalnum():
            raise ValueError('Username must be alphanumeric')
        return value

# Example usage
try:
    user = User(username='john_doe', age=17)
except ValidationError as e:
    print(e)

try:
    user = User(username='john@doe', age=20)
except ValidationError as e:
    print(e)

user = User(username='johndoe', age=20)
print(user)
```

In this example:
- The `User` model has two fields: `username` and `age`.
- The `check_age` method is a custom validator for the `age` field, ensuring that the age is at least 18.
- The `check_username` method is a custom validator for the `username` field, ensuring that the username is alphanumeric.

When creating an instance of the `User` model, the custom validators are called to enforce the additional validation rules. If the validation fails, a `ValidationError` is raised.

Note: The `@model_validator` decorator is not a built-in decorator in Pydantic. Instead, you should use the `@validator` decorator as shown in the example above. The `@validator` decorator is used to define custom validation logic for individual fields in a Pydantic model.

In [47]:
from pydantic import model_validator
from typing import Any

class Owner(BaseModel):
    name: str
    email: EmailStr

    @model_validator(mode='before')
    @classmethod
    def check_senstive_info_omitted(cls, data:Any) -> Any:
        if isinstance(data, dict):
            if 'password' in data:
                raise ValueError("Password field is not allowed")
            if 'card_number' in data:
                raise ValueError("Card number field is not allowed")    
            
            return data
        
    @model_validator(mode='after')
    def check_name_contains_space(self) -> Owner:
        if ' ' not in self.name:
            raise ValueError('Owner name must contain a space')
        return self 

In [48]:
try:
    owner = Owner(name="John Doe", email="zorba@gmail.com", Address="123, Flavor Street")
except Exception as e:
    print(e)

In [49]:
pprint(owner.model_dump())

{'email': 'zorba@gmail.com', 'name': 'John Doe'}


In [50]:
try:
    owner2 = Owner(name="John Doe", email="zorba@gmail.com", Address="123, Flavor Street",password="123456")
except Exception as e:
    print(e)

1 validation error for Owner
  Value error, Password field is not allowed [type=value_error, input_value={'name': 'John Doe', 'ema...', 'password': '123456'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error


In [51]:
print(owner2.model_dump())

{'name': 'John Doe', 'email': 'zorba@gmail.com'}


## More usage of Field


### Default Values

In [56]:
from pydantic import Field
from uuid import uuid4

class Owner(BaseModel):
    name: str = Field(default="John Doe")
    email: EmailStr = Field(default="name@example.com")
    id: str = Field(default_factory=lambda: uuid4().hex)


owner = Owner()
owner.model_dump()

{'name': 'John Doe',
 'email': 'name@example.com',
 'id': '8602b350a7e74bcbaa23aa3838e72545'}

### Aliases

In [58]:
class User(BaseModel):
    name: str = Field(...,alias="full_name")

u = User(full_name="John Doe")
print(u.model_dump())
print("Model Dump by alias")
print(u.model_dump(by_alias=True))


{'name': 'John Doe'}
Model Dump by alias
{'full_name': 'John Doe'}


# Field Constraints

In the `User` model, various field constraints are applied using the `Field` function from Pydantic. These constraints help enforce specific rules for the data validation. Here are the constraints used in the code:

In [62]:
from decimal import Decimal
from pydantic import BaseModel, Field

class User(BaseModel):
    name: str = Field(..., min_length=2, max_length=50)
    age: int = Field(..., gt=0, le=150)
    height: Decimal = Field(..., gt=0)
    weight: float = Field(..., ge=0)
    score: Decimal = Field(..., multiple_of=0.1)
    balance: Decimal = Field(..., max_digits=10, decimal_places=2)
    is_student: bool = Field(...)



### Constraints Explanation

- **name: str**
  - `min_length=2`: The name must be at least 2 characters long.
  - `max_length=50`: The name must be at most 50 characters long.

- **age: int**
  - `gt=0`: The age must be greater than 0.
  - `le=150`: The age must be less than or equal to 150.

- **height: Decimal**
  - `gt=0`: The height must be greater than 0.

- **weight: float**
  - `ge=0`: The weight must be greater than or equal to 0.

- **score: Decimal**
  - `multiple_of=0.1`: The score must be a multiple of 0.1.

- **balance: Decimal**
  - `max_digits=10`: The balance can have a maximum of 10 digits in total.
  - `decimal_places=2`: The balance can have up to 2 decimal places.

- **is_student: bool**
  - No additional constraints are applied, but the field is required (indicated by `...`).

These constraints ensure that the data provided to the `User` model adheres to the specified rules, helping to maintain data integrity and validity.

In [63]:
try:
    user = User(name="A", age=0, 
                height=-1, weight=-1, 
                score=0.2, 
                balance=100.123, 
                is_student="True")
except ValidationError as e:
    print(e)

5 validation errors for User
name
  String should have at least 2 characters [type=string_too_short, input_value='A', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/string_too_short
age
  Input should be greater than 0 [type=greater_than, input_value=0, input_type=int]
    For further information visit https://errors.pydantic.dev/2.10/v/greater_than
height
  Input should be greater than 0 [type=greater_than, input_value=-1, input_type=int]
    For further information visit https://errors.pydantic.dev/2.10/v/greater_than
weight
  Input should be greater than or equal to 0 [type=greater_than_equal, input_value=-1, input_type=int]
    For further information visit https://errors.pydantic.dev/2.10/v/greater_than_equal
balance
  Decimal input should have no more than 2 decimal places [type=decimal_max_places, input_value=100.123, input_type=float]
    For further information visit https://errors.pydantic.dev/2.10/v/decimal_max_places


The output indicates that there are multiple validation errors when trying to create an instance of the `User` model. Here's a breakdown of each error message:





### Errors

1. **name**
   - **String should have at least 2 characters**: The `name` field must have at least 2 characters.
   - **type=string_too_short**: The type of error is "string_too_short".
   - **input_value='A'**: The provided value is 'A'.
   - **input_type=str**: The type of the input is a string.

2. **age**
   - **Input should be greater than 0**: The `age` field must be greater than 0.
   - **type=greater_than**: The type of error is "greater_than".
   - **input_value=0**: The provided value is 0.
   - **input_type=int**: The type of the input is an integer.

3. **height**
   - **Input should be greater than 0**: The `height` field must be greater than 0.
   - **type=greater_than**: The type of error is "greater_than".
   - **input_value=-1**: The provided value is -1.
   - **input_type=int**: The type of the input is an integer.

4. **weight**
   - **Input should be greater than or equal to 0**: The `weight` field must be greater than or equal to 0.
   - **type=greater_than_equal**: The type of error is "greater_than_equal".
   - **input_value=-1**: The provided value is -1.
   - **input_type=int**: The type of the input is an integer.

5. **balance**
   - **Decimal input should have no more than 2 decimal places**: The `balance` field must have no more than 2 decimal places.
   - **type=decimal_max_places**: The type of error is "decimal_max_places".
   - **input_value=100.123**: The provided value is 100.123.
   - **input_type=float**: The type of the input is a float.

### Cause

Each error occurs because the provided input data does not meet the constraints defined in the `User` model.



In [66]:
from decimal import Decimal
from pydantic import BaseModel, Field

class User(BaseModel):
    name: str = Field(..., min_length=2, max_length=50)
    age: int = Field(..., gt=0, le=150)
    height: Decimal = Field(..., gt=0)
    weight: float = Field(..., ge=0)
    score: Decimal = Field(..., multiple_of=0.1)
    balance: Decimal = Field(..., max_digits=10, decimal_places=2)
    is_student: bool = Field(...)

# Correct input data
user_data = {
    'name': 'John',
    'age': 25,
    'height': Decimal('1.75'),
    'weight': 70.0,
    'score': Decimal('9.5'),
    'balance': Decimal('100.12'),
    'is_student': True
}

user = User(**user_data)
pprint(user.model_dump())

{'age': 25,
 'balance': Decimal('100.12'),
 'height': Decimal('1.75'),
 'is_student': True,
 'name': 'John',
 'score': Decimal('9.5'),
 'weight': 70.0}




By providing valid input data that adheres to the constraints, the validation errors will be resolved.

## Pydantic: Computed Fields

In Pydantic, computed fields are fields whose values are derived from other fields in the model. These fields are not directly set by the user but are computed based on the values of other fields. Computed fields can be implemented using the `@property` decorator or by defining methods that compute the values.

### Example

Here's an example of how to define computed fields in a Pydantic model:



In [67]:
from pydantic import BaseModel, Field, computed_field
from typing import List

class Item(BaseModel):
    name: str
    price: float
    quantity: int

class Order(BaseModel):
    items: List[Item]

    @computed_field
    @property
    def total_price(self) -> float:
        return sum(item.price * item.quantity for item in self.items)

    @computed_field
    @property
    def item_count(self) -> int:
        return sum(item.quantity for item in self.items)

# Example usage
order = Order(items=[
    Item(name='Apple', price=1.0, quantity=3),
    Item(name='Banana', price=0.5, quantity=5)
])

print(order.total_price)  # Output: 5.5
print(order.item_count)   # Output: 8

5.5
8




## Explanation

- **Item Model**: Represents an item with `name`, `price`, and `quantity` fields.
- **Order Model**: Represents an order containing a list of items.
  - **total_price**: A computed field that calculates the total price of all items in the order. It uses the `@property` decorator to define a method that sums the product of `price` and `quantity` for each item.
  - **item_count**: A computed field that calculates the total number of items in the order. It uses the `@property` decorator to define a method that sums the `quantity` of each item.

## Benefits

- **Encapsulation**: Computed fields encapsulate the logic for deriving values, making the model cleaner and easier to maintain.
- **Consistency**: Ensures that derived values are always consistent with the base data.
- **Readability**: Improves readability by providing meaningful properties that represent computed values.

Computed fields in Pydantic models help to keep the data consistent and encapsulate the logic for derived values, making the models more robust and easier to work with.

The `@computed_field` decorator in Pydantic is specifically designed for defining computed fields that are included in the model's schema and serialization. This is different from the `@property` decorator, which only defines a property on the class and does not include it in the model's schema or serialization by default.

Here's a comparison of what each decorator does in the context of Pydantic models:

### `@property`

- Defines a property on the class.
- The property is not included in the model's schema.
- The property is not included in the model's serialization (e.g., when converting to JSON).

### `@computed_field`

- Defines a computed field that is included in the model's schema.
- The computed field is included in the model's serialization.
- The computed field is automatically computed based on other fields in the model.


- **total_price_property**: Defined using the `@property` decorator. It is not included in the model's schema or serialization.
- **total_price_computed**: Defined using the `@computed_field` decorator. It is included in the model's schema and serialization.

### Benefits of `@computed_field`

- **Schema Inclusion**: The computed field is included in the model's schema, making it visible in generated documentation and validation.
- **Serialization**: The computed field is included in the model's serialization, ensuring that it is part of the output when converting the model to JSON or other formats.

Using `@computed_field` is beneficial when you need the computed field to be part of the model's schema and serialization, providing a more complete and consistent representation of the model's data.

> In Pydantic, you cannot directly apply a validator to a computed field using the `@validator` decorator because validators are designed to work with fields that are explicitly defined in the model. However, you can achieve similar functionality by using the `@root_validator` decorator, which allows you to validate the entire model after all fields have been populated, including computed fields.
